In [1]:
import torch
import torch.nn as nn

from efficientnet import EfficientNet_B0

from torchvision import transforms
from utils import *
from voc_dataloader import VocDataset, VOC_CLASSES

%load_ext autoreload
%autoreload 2


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std= [0.229, 0.224, 0.225])

# Use random crop and flip for training
train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224,scale = (0.3,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ])

test_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])

ds_train = VocDataset('../VOCdevkit_2007/VOC2007/','train',train_transform)
ds_val = VocDataset('../VOCdevkit_2007/VOC2007/','val',test_transform)
ds_test = VocDataset('../VOCdevkit_2007/VOC2007test/','test', test_transform)

E:\498_working\efficientN\voc_dataloader.py:109: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(names), np.array(labels).astype(np.float32), np.array(box_indices), label_order


In [8]:
num_epochs = 20
test_frequency = 5
batch_size = 36
# set a higher number of workers for better performance
num_workers = 8

train_loader = torch.utils.data.DataLoader(dataset=ds_train,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            pin_memory=True,
                                            num_workers=num_workers)

val_loader = torch.utils.data.DataLoader(dataset=ds_val,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            pin_memory=True,
                                            num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(dataset=ds_test,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               pin_memory=True,
                                               num_workers=num_workers)

Efficient Net B0

started 1:39

In [8]:
classifier = EfficientNet_B0().to(device)

criterion = nn.MultiLabelSoftMarginLoss()
#optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

classifier, train_losses, val_losses, train_mAPs, val_mAPs = \
    train(classifier, num_epochs, train_loader, val_loader, criterion, optimizer, test_frequency)



Starting epoch number 1
Loss for Training on Epoch 1 is 0.25039950013160706
-------  Class: aeroplane        AP:   0.0728  -------
-------  Class: bicycle          AP:   0.0619  -------
-------  Class: bird             AP:   0.1064  -------
-------  Class: boat             AP:   0.1138  -------
-------  Class: bottle           AP:   0.0712  -------
-------  Class: bus              AP:   0.0500  -------
-------  Class: car              AP:   0.1875  -------
-------  Class: cat              AP:   0.1526  -------
-------  Class: chair            AP:   0.1858  -------
-------  Class: cow              AP:   0.0393  -------
-------  Class: diningtable      AP:   0.1460  -------
-------  Class: dog              AP:   0.1318  -------
-------  Class: horse            AP:   0.0751  -------
-------  Class: motorbike        AP:   0.0698  -------
-------  Class: person           AP:   0.4189  -------
-------  Class: pottedplant      AP:   0.0490  -------
-------  Class: sheep            AP:   0.067

KeyboardInterrupt: 

In [9]:
from efficientnet_pytorch import EfficientNet
ori = EfficientNet.from_pretrained('efficientnet-b0')
ori._fc = nn.Linear(1280,21)
ori = ori.to(device)

Loaded pretrained weights for efficientnet-b0


In [10]:
criterion = nn.MultiLabelSoftMarginLoss()
#optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(ori.parameters(), lr=1e-4)

ori, train_losses, val_losses, train_mAPs, val_mAPs = \
    train(ori, num_epochs, train_loader, val_loader, criterion, optimizer, test_frequency)

Starting epoch number 1
Loss for Training on Epoch 1 is 0.4731805622577667
-------  Class: aeroplane        AP:   0.0923  -------
-------  Class: bicycle          AP:   0.1192  -------
-------  Class: bird             AP:   0.1957  -------
-------  Class: boat             AP:   0.0681  -------
-------  Class: bottle           AP:   0.0810  -------
-------  Class: bus              AP:   0.0485  -------
-------  Class: car              AP:   0.2938  -------
-------  Class: cat              AP:   0.1343  -------
-------  Class: chair            AP:   0.2284  -------
-------  Class: cow              AP:   0.0297  -------
-------  Class: diningtable      AP:   0.1066  -------
-------  Class: dog              AP:   0.3090  -------
-------  Class: horse            AP:   0.1550  -------
-------  Class: motorbike        AP:   0.1064  -------
-------  Class: person           AP:   0.8242  -------
-------  Class: pottedplant      AP:   0.0512  -------
-------  Class: sheep            AP:   0.0153

Linear(in_features=1280, out_features=1000, bias=True)

In [11]:
ori



EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=